In [1]:
import sqlite3
import numpy as np
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from xgboost import plot_importance
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
import pickle

In [2]:
cnx = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes",cnx)

In [3]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [4]:
target = df.pop('overall_rating')

In [5]:
df.shape

(183978, 41)

In [6]:
target.head()

0    67.0
1    67.0
2    62.0
3    61.0
4    61.0
Name: overall_rating, dtype: float64

In [7]:
target.isnull().values.sum()

836

In [8]:
target.describe()

count    183142.000000
mean         68.600015
std           7.041139
min          33.000000
25%          64.000000
50%          69.000000
75%          73.000000
max          94.000000
Name: overall_rating, dtype: float64

In [9]:
plt.hist(target, 30, range=(33,94))

<IPython.core.display.Javascript object>

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


(array([7.0000e+00, 6.0000e+00, 2.0000e+01, 6.5000e+01, 9.4000e+01,
        1.4200e+02, 2.9400e+02, 5.2600e+02, 1.2510e+03, 1.8450e+03,
        2.5780e+03, 4.0870e+03, 6.1890e+03, 9.6500e+03, 1.3745e+04,
        1.8366e+04, 2.0310e+04, 2.0773e+04, 1.9382e+04, 1.8784e+04,
        1.5915e+04, 1.1254e+04, 7.5250e+03, 4.9470e+03, 2.8290e+03,
        1.4590e+03, 7.4800e+02, 2.2800e+02, 8.4000e+01, 3.9000e+01]),
 array([33.        , 35.03333333, 37.06666667, 39.1       , 41.13333333,
        43.16666667, 45.2       , 47.23333333, 49.26666667, 51.3       ,
        53.33333333, 55.36666667, 57.4       , 59.43333333, 61.46666667,
        63.5       , 65.53333333, 67.56666667, 69.6       , 71.63333333,
        73.66666667, 75.7       , 77.73333333, 79.76666667, 81.8       ,
        83.83333333, 85.86666667, 87.9       , 89.93333333, 91.96666667,
        94.        ]),
 <a list of 30 Patch objects>)

In [10]:
y = target.fillna(target.mean())

In [11]:
y.isnull().values.any()

False

In [12]:
df.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'potential',
       'preferred_foot', 'attacking_work_rate', 'defensive_work_rate',
       'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving',
       'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes'],
      dtype='object')

In [13]:
for col in df.columns:
    unique_cat = len(df[col].unique())
    print("{col}-->{unique_cat}..{typ}".format(col=col, unique_cat=unique_cat, typ=df[col].dtype))

id-->183978..int64
player_fifa_api_id-->11062..int64
player_api_id-->11060..int64
date-->197..object
potential-->57..float64
preferred_foot-->3..object
attacking_work_rate-->9..object
defensive_work_rate-->20..object
crossing-->96..float64
finishing-->98..float64
heading_accuracy-->97..float64
short_passing-->96..float64
volleys-->94..float64
dribbling-->98..float64
curve-->93..float64
free_kick_accuracy-->98..float64
long_passing-->96..float64
ball_control-->94..float64
acceleration-->87..float64
sprint_speed-->86..float64
agility-->82..float64
reactions-->79..float64
balance-->82..float64
shot_power-->97..float64
jumping-->80..float64
stamina-->85..float64
strength-->83..float64
long_shots-->97..float64
aggression-->92..float64
interceptions-->97..float64
positioning-->96..float64
vision-->98..float64
penalties-->95..float64
marking-->96..float64
standing_tackle-->96..float64
sliding_tackle-->95..float64
gk_diving-->94..float64
gk_handling-->91..float64
gk_kicking-->98..float64
gk_po

In [14]:
df_dummy = pd.get_dummies(df, columns=['preferred_foot','attacking_work_rate','defensive_work_rate'])
df_dummy.head()

,id,player_fifa_api_id,player_api_id,date,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,defensive_work_rate_9,defensive_work_rate__0,defensive_work_rate_ean,defensive_work_rate_es,defensive_work_rate_high,defensive_work_rate_low,defensive_work_rate_medium,defensive_work_rate_o,defensive_work_rate_ormal,defensive_work_rate_tocky
0,1,218353,505942,2016-02-18 00:00:00,71.0,49.0,44.0,71.0,61.0,44.0,...,0,0,0,0,0,0,1,0,0,0
1,2,218353,505942,2015-11-19 00:00:00,71.0,49.0,44.0,71.0,61.0,44.0,...,0,0,0,0,0,0,1,0,0,0
2,3,218353,505942,2015-09-21 00:00:00,66.0,49.0,44.0,71.0,61.0,44.0,...,0,0,0,0,0,0,1,0,0,0
3,4,218353,505942,2015-03-20 00:00:00,65.0,48.0,43.0,70.0,60.0,43.0,...,0,0,0,0,0,0,1,0,0,0
4,5,218353,505942,2007-02-22 00:00:00,65.0,48.0,43.0,70.0,60.0,43.0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
X = df_dummy.drop(['id','date'],axis = 1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=40)

In [17]:
imput = SimpleImputer()
X_train = imput.fit_transform(X_train)
X_test = imput.fit_transform(X_test)

In [18]:
model = XGBRegressor()
model.fit(X_train,y_train)
print(model.feature_importances_)

[1.1780912e-02 7.3718647e-03 1.6630439e-01 1.3897501e-02 7.8724781e-03
 2.4805792e-02 1.4548133e-02 1.2451592e-03 1.4464436e-02 9.7570632e-04
 8.3896681e-04 2.7907845e-03 5.0738730e-02 3.2936821e-03 6.0198172e-03
 1.0870639e-03 3.4993333e-01 1.0071513e-03 2.2891887e-02 2.3231537e-03
 5.3412840e-03 1.8412376e-02 5.6142746e-03 6.0821944e-03 1.5814187e-02
 1.3969570e-02 2.7534734e-03 1.2685431e-03 2.3455620e-02 5.4394376e-02
 7.3360028e-03 3.9118439e-02 2.0295044e-02 7.6389699e-03 4.5740783e-02
 1.1277276e-02 1.2072143e-03 0.0000000e+00 1.1350332e-03 5.9814384e-04
 1.5476078e-03 4.6846524e-04 1.9956264e-03 2.1882531e-04 0.0000000e+00
 5.9032341e-04 0.0000000e+00 6.8088766e-04 1.0040329e-03 0.0000000e+00
 0.0000000e+00 0.0000000e+00 9.3609677e-04 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.2660827e-03 0.0000000e+00 0.0000000e+00 2.5256143e-03
 1.3370307e-03 5.8614725e-04 1.1994631e-03 0.0000000e+00 0.0000000e+00]


In [19]:
selection = SelectFromModel(model, threshold=0.01,prefit=True)
select_X_train = selection.transform(X_train)
select_X_test = selection.transform(X_test)

In [20]:
select_X_train.shape

(137983, 18)

1. Linear Regression

In [21]:
pipe = make_pipeline(StandardScaler(),LinearRegression())
cv = ShuffleSplit(random_state=0)
param_grid = {'linearregression__n_jobs':[-1]}
grid = GridSearchCV(pipe,param_grid=param_grid,cv=cv)
grid.fit(select_X_train,y_train)

GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=0, test_size=None, train_size=None),
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'linearregression__n_jobs': [-1]})

In [22]:
grid.best_params_

{'linearregression__n_jobs': -1}

In [23]:
lin_reg = pickle.dumps(grid)

2. Decision Tree

In [24]:
pipe = make_pipeline(StandardScaler(),DecisionTreeRegressor(criterion='mse',random_state=0))
cv = ShuffleSplit(n_splits=10,random_state=40)
param_grid = {'decisiontreeregressor__max_depth':[3,5,7,9,13]}
grid = GridSearchCV(pipe,param_grid=param_grid,cv=cv)
grid.fit(select_X_train,y_train)

GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=40, test_size=None, train_size=None),
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('decisiontreeregressor',
                                        DecisionTreeRegressor(random_state=0))]),
             param_grid={'decisiontreeregressor__max_depth': [3, 5, 7, 9, 13]})

In [25]:
grid.best_params_

{'decisiontreeregressor__max_depth': 13}

In [26]:
Dectree_reg = pickle.dumps(grid)

3. XGBOOST Regressor

In [28]:
pipe = make_pipeline(StandardScaler(),XGBRegressor(n_estimators = 500, random_state = 40))
cv = ShuffleSplit(n_splits=10,random_state=0)
param_grid = {'xgbregressor__max_depth':[5,7],'xgbregressor__learning_rate':[0.1,0.3]}
grid = GridSearchCV(pipe,param_grid=param_grid,cv=cv,n_jobs=-1)
grid.fit(select_X_train,y_train)

GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=0, test_size=None, train_size=None),
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('xgbregressor',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     gamma=None, gpu_id=None,
                                                     importance_type='gain',
                                                     interaction_constraints=None,...
                                                     min_child_weight=None,
                                                     missing=nan,
                          

In [29]:
grid.best_params_

{'xgbregressor__learning_rate': 0.3, 'xgbregressor__max_depth': 7}

In [30]:
xgbreg = pickle.dumps(grid)

In [31]:
lin_reg = pickle.loads(lin_reg)
Dectree_reg = pickle.loads(Dectree_reg)
xgbreg = pickle.loads(xgbreg)

In [32]:
print("""Linear Regressor accuracy is {lin}
DecisionTree Regressor accuracy is {Dec}
XGBoost regressor accuracy is {xgb}""".format(lin=lin_reg.score(select_X_test, y_test),
                                                       Dec=Dectree_reg.score(select_X_test, y_test),
                                                       xgb=xgbreg.score(select_X_test, y_test)))

Linear Regressor accuracy is 0.8421252537524467
DecisionTree Regressor accuracy is 0.9320586184529094
XGBoost regressor accuracy is 0.9792302962188885


By accuracy comparision performed above we can say hear that XGBoost regressor gives better result than any other model. and it can predict the target function with approx 98% accuracy.